## Labeling

In [57]:
import pandas as pd
import nltk
from textblob import TextBlob

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [58]:
data = pd.read_csv("../../preprocessing-data/rewind-2015/translate_comments.csv")
data

,text,tweet_english
0,mana mbak sasa stepenson,where is mbak sasa stepenson
1,bang eka please buatin mixnya lagu bad blod ca...,bang eka please make the mix of the song bad b...
2,watch me whip naenae,watch me whip naenae
3,10 nama lgunya apa,10 what 'S the name OF the lagoons
4,aulion paling hebat,the ultimate aulion
...,...,...
95,dlu fun now just giveaway buat indo makin hancur,etc. fun now just giveaway for indo getting ru...
96,awesome col,awesome col
97,can wait to go to indonesia this country is am...,can wait to go to indonesia this country is am...
98,ini baik dapat ku,this is good can i


## Analisis dengan metode Text Blob

In [59]:
data_comments = list(data["tweet_english"])
polaritas = 0

status = []
total_positif = total_netral = total_negatif = total = 0

for i, comment in enumerate(data_comments):
  analysis = TextBlob(comment)
  polaritas += analysis.polarity

  if analysis.sentiment.polarity > 0.0:
    total_positif += 1
    status.append("Positif")
  elif analysis.sentiment.polarity == 0.0:
    total_netral += 1
    status.append("Netral")
  else:
    total_negatif += 1
    status.append("Negatif")
  
  total += 1

In [60]:
print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'Total Data = {total}')

Hasil Analisis Data:
Positif = 40
Netral = 49
Negatif = 11
Total Data = 100


In [61]:
# Data sentimen
data_sentimen = [total_positif, total_netral, total_negatif, total]

# Membuat DataFrame dari data sentimen
df = pd.DataFrame([data_sentimen], columns=['total_positif', 'total_netral', 'total_negatif', 'total'])

# Menyimpan DataFrame ke dalam file CSV
df.to_csv('data_sentimen_textblob.csv', index=False)

In [62]:
data["analisis_textblob"] = status
data

,text,tweet_english,analisis_textblob
0,mana mbak sasa stepenson,where is mbak sasa stepenson,Netral
1,bang eka please buatin mixnya lagu bad blod ca...,bang eka please make the mix of the song bad b...,Positif
2,watch me whip naenae,watch me whip naenae,Netral
3,10 nama lgunya apa,10 what 'S the name OF the lagoons,Netral
4,aulion paling hebat,the ultimate aulion,Netral
...,...,...,...
95,dlu fun now just giveaway buat indo makin hancur,etc. fun now just giveaway for indo getting ru...,Positif
96,awesome col,awesome col,Positif
97,can wait to go to indonesia this country is am...,can wait to go to indonesia this country is am...,Positif
98,ini baik dapat ku,this is good can i,Positif


## Analisis dengan metode Naive Bayes Classifier

In [63]:
data

,text,tweet_english,analisis_textblob
0,mana mbak sasa stepenson,where is mbak sasa stepenson,Netral
1,bang eka please buatin mixnya lagu bad blod ca...,bang eka please make the mix of the song bad b...,Positif
2,watch me whip naenae,watch me whip naenae,Netral
3,10 nama lgunya apa,10 what 'S the name OF the lagoons,Netral
4,aulion paling hebat,the ultimate aulion,Netral
...,...,...,...
95,dlu fun now just giveaway buat indo makin hancur,etc. fun now just giveaway for indo getting ru...,Positif
96,awesome col,awesome col,Positif
97,can wait to go to indonesia this country is am...,can wait to go to indonesia this country is am...,Positif
98,ini baik dapat ku,this is good can i,Positif


In [64]:
dataset = data.drop(["text"], axis=1, inplace=False)
dataset = [tuple(x) for x in dataset.to_records(index=False)]

dataset

[('where is mbak sasa stepenson', 'Netral'),
 ('bang eka please make the mix of the song bad blod mix sambaladonya is really good',
  'Positif'),
 ('watch me whip naenae', 'Netral'),
 ("10 what 'S the name OF the lagoons", 'Netral'),
 ('the ultimate aulion', 'Netral'),
 ('sory min 4years late', 'Negatif'),
 ('what is the name of the song pas slowmotion', 'Netral'),
 ('damn surprise there is yongleklangsung banting ya hp gw', 'Netral'),
 ('the look of time laughing at itself kece abis', 'Netral'),
 ('aulion is really great', 'Positif'),
 ('so cool emg it gives me goosebumps', 'Positif'),
 ('2016 invited motovloger donk', 'Netral'),
 ('what is the name of the 14 minute music', 'Netral'),
 ('holy shiet just realize thats rich chiga fux 5', 'Positif'),
 ('2016 invites motomobi ridwan hanif and motovloger2', 'Netral'),
 ('pink shirt was sopo yo wkwkwk', 'Negatif'),
 ('2020 spa msih nton mana ytrewind 2019 nya ktnya 2019 bnyak yutuber',
  'Netral'),
 ('watch rewind 2021 directly here', 'Posi

In [65]:
import random

set_positif = []
set_negatif = []
set_netral = []

for n in dataset:
    if(n[1] == "Positif"):
        set_positif.append(n)
    elif(n[1] == "Negatif"):
        set_negatif.append(n)
    else:
        set_netral.append(n)

set_positif = random.sample(set_positif, k=int(len(set_positif)/2))
set_negatif = random.sample(set_negatif, k=int(len(set_positif)/2))
set_netral = random.sample(set_netral, k=int(len(set_positif)/2))

train = set_positif + set_negatif + set_netral

train_set = []

for n in train:
    train_set.append(n)
    

In [66]:
from textblob.classifiers import NaiveBayesClassifier

cl = NaiveBayesClassifier(train_set)
print("Akurasi Test : ", cl.accuracy(dataset))

Akurasi Test :  0.59


## Labeling Naive Bayes

In [67]:
data_comments = list(data["tweet_english"])
polaritas = 0

status = []
total_positif = total_netral = total_negatif = total = 0

for i, comment in enumerate(data_comments):
    analysis = TextBlob(comment, classifier=cl)

    if analysis.classify() == "Positif":
        total_positif += 1
    elif analysis.classify() == "Negatif":
        total_negatif += 1
    else:
        total_netral += 1
    
    status.append(analysis.classify())
    total += 1

In [68]:
print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'Total Data = {total}')

Hasil Analisis Data:
Positif = 75
Netral = 13
Negatif = 12
Total Data = 100


In [69]:
status = pd.DataFrame({"analisis_naive_bayes": status})

data["analisis_naive_bayes"] = status
data

,text,tweet_english,analisis_textblob,analisis_naive_bayes
0,mana mbak sasa stepenson,where is mbak sasa stepenson,Netral,Positif
1,bang eka please buatin mixnya lagu bad blod ca...,bang eka please make the mix of the song bad b...,Positif,Positif
2,watch me whip naenae,watch me whip naenae,Netral,Positif
3,10 nama lgunya apa,10 what 'S the name OF the lagoons,Netral,Netral
4,aulion paling hebat,the ultimate aulion,Netral,Positif
...,...,...,...,...
95,dlu fun now just giveaway buat indo makin hancur,etc. fun now just giveaway for indo getting ru...,Positif,Positif
96,awesome col,awesome col,Positif,Positif
97,can wait to go to indonesia this country is am...,can wait to go to indonesia this country is am...,Positif,Positif
98,ini baik dapat ku,this is good can i,Positif,Positif


In [70]:
# Data sentimen
data_sentimen = [total_positif, total_netral, total_negatif, total]

# Membuat DataFrame dari data sentimen
df = pd.DataFrame([data_sentimen], columns=['total_positif', 'total_netral', 'total_negatif', 'total'])

# Menyimpan DataFrame ke dalam file CSV
df.to_csv('data_sentimen_naive_bayes.csv', index=False)

In [71]:
data = data.drop("tweet_english", axis=1)
data

,text,analisis_textblob,analisis_naive_bayes
0,mana mbak sasa stepenson,Netral,Positif
1,bang eka please buatin mixnya lagu bad blod ca...,Positif,Positif
2,watch me whip naenae,Netral,Positif
3,10 nama lgunya apa,Netral,Netral
4,aulion paling hebat,Netral,Positif
...,...,...,...
95,dlu fun now just giveaway buat indo makin hancur,Positif,Positif
96,awesome col,Positif,Positif
97,can wait to go to indonesia this country is am...,Positif,Positif
98,ini baik dapat ku,Positif,Positif


In [72]:
data.to_csv("analysis_comments.csv", index=False)